<a href="https://colab.research.google.com/github/ShashankKamath/DecisionTree_ML/blob/master/DT_q2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
#  https://github.com/random-forests/tutorials/blob/master/decision_tree.ipynb

In [1]:
from google.colab import files
uploaded = files.upload()


Saving hw2_question1.csv to hw2_question1.csv


Splitting the data into Train and Test


In [2]:
import pandas as pd
import numpy as np
data = pd.read_csv("hw2_extra.csv")
train_set=data.values.tolist()
header=["Temperature","Humidity","Sky_Condition","Rainy"]

FileNotFoundError: ignored

In [3]:
data.head()

,Temperature,Humidity,Sky_condition,Output
0,hot,high,cloudy,1
1,hot,high,cloudy,1
2,hot,high,cloudy,1
3,hot,high,cloudy,1
4,hot,high,cloudy,1


In [0]:
from sklearn.utils import shuffle
data = pd.read_csv("hw2_question1.csv")
header=["Thickness", "Cellsize", "Cellshape", "Adhesion","Ecellsize","Nuclei","Chromatin","Nucleoli","Mitosis","Class"]
data.columns=header
data=shuffle(data)
data=data.reset_index(drop=True)
#   data = data.sample(frac=1).reset_index(drop=True)
train_data=pd.DataFrame()
test_data=pd.DataFrame()
class_2_count=[295,148]
class_4_count=[159,80]
for i in range(0,data.shape[0]):
    if(data.loc[i,'Class']==2 and class_2_count[0]<=295 and class_2_count[0]>0):
        train_data=train_data.append(data.iloc[i])
        class_2_count[0]-=1
    if(data.loc[i,'Class']==4 and class_4_count[0]<=159 and class_4_count[0]>0):
        train_data=train_data.append(data.iloc[i])
        class_4_count[0]-=1
    if(data.loc[i,'Class']==2 and class_2_count[1]<=148 and class_2_count[1]>0):
        test_data=test_data.append(data.iloc[i])
        class_2_count[1]-=1
    if(data.loc[i,'Class']==4 and class_4_count[1]<=80 and class_4_count[1]>0):
        test_data=test_data.append(data.iloc[i])
        class_4_count[1]-=1
train_data=train_data[header]
test_data=test_data[header]
data_set=data.values.tolist()
train_set=train_data.values.tolist()
test_set=test_data.values.tolist()

In [4]:
def unique_vals(rows, col):
    """Find the unique values for a column in a dataset."""
    return set([row[col] for row in rows])
unique_vals(train_set, 0)

{1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, 9.0, 10.0}

In [6]:
def class_counts(rows):
    """Counts the number of each type of example in a dataset."""
    counts = {}  # a dictionary of label -> count.
    for row in rows:
        # in our dataset format, the label is always the last column
        label = row[-1]
        if label not in counts:
            counts[label] = 0
        counts[label] += 1
    return counts
  
class_counts(train_set)

{2.0: 295, 4.0: 159}

In [7]:
def is_numeric(value):
    """Test if a value is numeric."""
    return isinstance(value, int) or isinstance(value, float)
is_numeric(7)

True

In [0]:
class Question:
    """A Question is used to partition a dataset.

    This class just records a 'column number' (e.g., 0 for Color) and a
    'column value' (e.g., Green). The 'match' method is used to compare
    the feature value in an example to the feature value stored in the
    question. See the demo below.
    """

    def __init__(self, column, value):
        self.column = column
        self.value = value

    def match(self, example):
        # Compare the feature value in an example to the
        # feature value in this question.
        val = example[self.column]
        if is_numeric(val):
            return val >= self.value
        else:
            return val == self.value

    def __repr__(self):
        # This is just a helper method to print
        # the question in a readable format.
        condition = "=="
        if is_numeric(self.value):
            condition = ">="
        return "Is %s %s %s?" % (header[self.column], condition, str(self.value))


In [9]:
q = Question(6,7)
example = train_set[0]
q.match(example)


True

In [0]:
def partition(rows, question):
    """Partitions a dataset.

    For each row in the dataset, check if it matches the question. If
    so, add it to 'true rows', otherwise, add it to 'false rows'.
    """
    true_rows, false_rows = [], []
    for row in rows:
        if question.match(row):
            true_rows.append(row)
        else:
            false_rows.append(row)
    return true_rows, false_rows

In [0]:
# true_rows, false_rows = partition(train_set, Question(0, 2))
# true_rows

This will give all values less than two in the first column

In [0]:
def gini(rows):
    """Calculate the Gini Impurity for a list of rows.

    There are a few different ways to do this, I thought this one was
    the most concise. See:
    https://en.wikipedia.org/wiki/Decision_tree_learning#Gini_impurity
    """
    counts = class_counts(rows)
    impurity = 1
    for lbl in counts:
        prob_of_lbl = counts[lbl] / float(len(rows))
        impurity -= prob_of_lbl**2
    return impurity 

In [0]:
def info_gain(left, right, current_uncertainty):
    """Information Gain.

    The uncertainty of the starting node, minus the weighted impurity of
    two child nodes.
    """
    p = float(len(left)) / (len(left) + len(right))
    return current_uncertainty - p * gini(left) - (1 - p) * gini(right)

In [13]:
current_uncertainty = gini(train_set)
current_uncertainty

0.49499999999999994

In [0]:
def find_best_split(rows):
    """Find the best question to ask by iterating over every feature / value
    and calculating the information gain."""
    best_gain = 0  # keep track of the best information gain
    best_question = None  # keep train of the feature / value that produced it
    current_uncertainty = gini(rows)
    n_features = len(rows[0]) - 1  # number of columns

    for col in range(n_features):  # for each feature

        values = set([row[col] for row in rows])  # unique values in the column

        for val in values:  # for each value

            question = Question(col, val)

            # try splitting the dataset
            true_rows, false_rows = partition(rows, question)

            # Skip this split if it doesn't divide the
            # dataset.
            if len(true_rows) == 0 or len(false_rows) == 0:
                continue

            # Calculate the information gain from this split
            gain = info_gain(true_rows, false_rows, current_uncertainty)

            # You actually can use '>' instead of '>=' here
            # but I wanted the tree to look a certain way for our
            # toy dataset.
#             print("Question and gain:",question,gain)
            if gain >= best_gain:
                best_gain, best_question = gain, question
    print("Best Question: ",best_question)
    print("Best Gain: ", best_gain)
    return best_gain, best_question

In [25]:
best_gain, best_question = find_best_split(train_set)
best_question, best_gain

Question and gain: None 0
Question and gain: Is Temperature == hot? 0.03124999999999989
Question and gain: Is Temperature == cool? 0.031249999999999917
Question and gain: Is Humidity == low? 0.031249999999999917
Question and gain: Is Humidity == low? 0.031249999999999917
Question and gain: Is Sky_Condition == clear? 0.06124999999999997
Best Question:  Is Sky_Condition == cloudy?
Best Gain:  0.06124999999999997


(Is Sky_Condition == cloudy?, 0.06124999999999997)

In [0]:
class Leaf:
    """A Leaf node classifies data.

    This holds a dictionary of class (e.g., "Apple") -> number of times
    it appears in the rows from the training data that reach this leaf.
    """

    def __init__(self, rows):
        self.predictions = class_counts(rows)

In [0]:
class Decision_Node:
    """A Decision Node asks a question.

    This holds a reference to the question, and to the two child nodes.
    """

    def __init__(self,
                 question,
                 true_branch,
                 false_branch):
        self.question = question
        self.true_branch = true_branch
        self.false_branch = false_branch

In [0]:
def print_tree(node, spacing=""):
    """World's most elegant tree printing function."""

    # Base case: we've reached a leaf
    if isinstance(node, Leaf):
        print (spacing + "Predict", node.predictions)
        return

    # Print the question at this node
    print (spacing + str(node.question))

    # Call this function recursively on the true branch
    print (spacing + '--> True:')
    print_tree(node.true_branch, spacing + "  ")

    # Call this function recursively on the false branch
    print (spacing + '--> False:')
    print_tree(node.false_branch, spacing + "  ")

In [0]:
# def build_tree(rows,running_node,running_depth,max_depth):
#     depth_nodes_total=math.pow(2,running_depth)#number of nodes in that depth
#     temp_node=0
#     #Should check the running node is in which depth and current node number 
#     while(running_node>-1):
      
      
#       running_node-=1
      
    
    
    
#     gain, question = find_best_split(rows)
#     if gain == 0:
#         return Leaf(rows)
# #     true_rows, false_rows = partition(rows, question)
   
#     if temp_node<running_node:
#       #make decision node and true branch and false branch
#       true_rows, false_rows = partition(rows, question)
#       true_branch = build_tree(true_rows,max_depth)
#       false_branch = build_tree(false_rows,max_depth)
#       return Decision_Node(question, true_branch, false_branch)

#     if temp_node==depth_nodes_total:
#         true_rows, false_rows = partition(rows, question)
#         true_branch=Leaf(true_rows)
#         false_branch=Leaf(false_rows)
#         return Decision_Node(question, true_branch, false_branch)      
    
# #     if(max_depth>-1):
# #       max_depth-=1
      
# #       true_branch = build_tree(true_rows,max_depth)


# #       false_branch = build_tree(false_rows,max_depth)


# #       return Decision_Node(question, true_branch, false_branch)

In [0]:
# def build_tree(rows,num_nodes):
#   gain, question = find_best_split(rows)
#   if gain == 0:
#       return Leaf(rows)
#   while(num_nodes>0):
#     true_rows, false_rows = partition(rows, question)
#     Decision_Node(question, true_branch, false_branch)
#     num_nodes-=1
#   if num_nodes==1:
#       true_rows, false_rows = partition(rows, question)
#       true_branch=Leaf(true_rows)
#       false_branch=Leaf(false_rows)
#   return Decision_Node(question, true_branch, false_branch)

In [0]:
def build_tree(rows,max_depth,depth):
    """Builds the tree.

    Rules of recursion: 1) Believe that it works. 2) Start by checking
    for the base case (no further information gain). 3) Prepare for
    giant stack traces.
    """

    # Try partitioing the dataset on each of the unique attribute,
    # calculate the information gain,
    # and return the question that produces the highest gain.
    print("Depth: ",depth)
    gain, question = find_best_split(rows)
  
    # Base case: no further info gain
    # Since we can ask no further questions,
    # we'll return a leaf.
    if gain == 0 or depth>=max_depth:
        print("Created Leaf Node")
        depth=depth-1
        return Leaf(rows)
    if depth<max_depth:
      true_rows, false_rows = partition(rows, question)
      print("Creating True Branch")
      true_branch = build_tree(true_rows,max_depth,depth+1)

      print("Creating False Branch")
      false_branch = build_tree(false_rows,max_depth,depth+1)

      # Return a Question node.
      # This records the best feature / value to ask at this point,
      # as well as the branches to follow
      # dependingo on the answer.
      depth=depth-1
      return Decision_Node(question, true_branch, false_branch)

In [0]:
#working
# def build_tree(rows):
#     """Builds the tree.

#     Rules of recursion: 1) Believe that it works. 2) Start by checking
#     for the base case (no further information gain). 3) Prepare for
#     giant stack traces.
#     """

#     # Try partitioing the dataset on each of the unique attribute,
#     # calculate the information gain,
#     # and return the question that produces the highest gain.
#     gain, question = find_best_split(rows)

#     # Base case: no further info gain
#     # Since we can ask no further questions,
#     # we'll return a leaf.
#     if gain == 0:
#         print("Created Leaf Node")
#         return Leaf(rows)

#     # If we reach here, we have found a useful feature / value
#     # to partition on.
#     true_rows, false_rows = partition(rows, question)

#     # Recursively build the true branch.
#     print("Creating True Branch")
#     true_branch = build_tree(true_rows)

#     # Recursively build the false branch.
#     print("Creating False Branch")
#     false_branch = build_tree(false_rows)

#     # Return a Question node.
#     # This records the best feature / value to ask at this point,
#     # as well as the branches to follow
#     # dependingo on the answer.
#     return Decision_Node(question, true_branch, false_branch)

In [26]:
import math
max_depth=2
num_nodes_total=(math.pow(2,max_depth+1)-1)
current_node=0
current_depth=0
rows=train_set
if(current_depth<=max_depth)
  while(current_node<=num_nodes_total):
    my_tree = build_tree(rows,current_node,current_depth,max_depth)
    for row in test_set:
      print ("Actual: %s. Predicted: %s" %
             (row[-1], print_leaf(classify(row, my_tree))))
    current_node+=1
  current_depth+=1
# my_tree=build_tree(train_set,num_nodes)
print_tree(my_tree)

SyntaxError: ignored

In [50]:
import math
max_depth=1
# num_nodes=(math.pow(2,max_depth+1)-1)
my_tree=build_tree(train_set,max_depth,0)
print_tree(my_tree)

Depth:  0
Best Question:  Is Cellsize >= 3.0?
Best Gain:  0.34040346770471613
Creating True Branch
Depth:  1
Best Question:  Is Cellshape >= 3.0?
Best Gain:  0.0637553286138829
Created Leaf Node
Creating False Branch
Depth:  1
Best Question:  Is Nuclei >= 6.0?
Best Gain:  0.034568181818181797
Created Leaf Node
Is Cellsize >= 3.0?
--> True:
  Predict {4.0: 152, 2.0: 22}
--> False:
  Predict {2.0: 273, 4.0: 7}


Question and gain: Is Temperature == hot? 0.03124999999999989
Question and gain: Is Temperature == cool? 0.031249999999999917
Question and gain: Is Humidity == low? 0.031249999999999917
Question and gain: Is Humidity == high? 0.03124999999999989
Question and gain: Is Sky_Condition == clear? 0.06124999999999997
Question and gain: Is Sky_Condition == cloudy? 0.06124999999999997
Best Question:  Is Sky_Condition == cloudy?
Best Gain:  0.06124999999999997
Creating True Branch
Question and gain: Is Temperature == hot? 0.03125
Question and gain: Is Temperature == cool? 0.03125
Question and gain: Is Humidity == low? 0.06125000000000011
Question and gain: Is Humidity == high? 0.06125000000000011
Best Question:  Is Humidity == high?
Best Gain:  0.06125000000000011
Creating True Branch
Question and gain: Is Temperature == hot? 0.01999999999999985
Question and gain: Is Temperature == cool? 0.01999999999999985
Best Question:  Is Temperature == cool?
Best Gain:  0.01999999999999985
Creating True Branch
Best Question:  None
Best Gain:  0
Created Leaf Node
Creating False Branch
Best Question:  None
Best Gain:  0
Created Leaf Node
Creating False Branch
Question and gain: Is Temperature == hot? 0.0449999999999999
Question and gain: Is Temperature == cool? 0.04499999999999993
Best Question:  Is Temperature == cool?
Best Gain:  0.04499999999999993
Creating True Branch
Best Question:  None
Best Gain:  0
Created Leaf Node
Creating False Branch
Best Question:  None
Best Gain:  0
Created Leaf Node
Creating False Branch
Question and gain: Is Temperature == hot? 0.03124999999999989
Question and gain: Is Temperature == cool? 0.03124999999999989
Question and gain: Is Humidity == low? 0.01125000000000001
Question and gain: Is Humidity == high? 0.01125000000000001
Best Question:  Is Temperature == cool?
Best Gain:  0.03124999999999989
Creating True Branch
Question and gain: Is Humidity == low? 0.0050000000000002265
Question and gain: Is Humidity == high? 0.0050000000000002265
Best Question:  Is Humidity == high?
Best Gain:  0.0050000000000002265
Creating True Branch
Best Question:  None
Best Gain:  0
Created Leaf Node
Creating False Branch
Best Question:  None
Best Gain:  0
Created Leaf Node
Creating False Branch
Question and gain: Is Humidity == low? 0.019999999999999907
Question and gain: Is Humidity == high? 0.019999999999999934
Best Question:  Is Humidity == high?
Best Gain:  0.019999999999999934
Creating True Branch
Best Question:  None
Best Gain:  0
Created Leaf Node
Creating False Branch
Best Question:  None
Best Gain:  0
Created Leaf Node
Is Sky_Condition == cloudy?
--> True:
  Is Humidity == high?
  --> True:
    Is Temperature == cool?
    --> True:
      Predict {1: 7, 0: 3}
    --> False:
      Predict {1: 9, 0: 1}
  --> False:
    Is Temperature == cool?
    --> True:
      Predict {1: 3, 0: 7}
    --> False:
      Predict {1: 6, 0: 4}
--> False:
  Is Temperature == cool?
  --> True:
    Is Humidity == high?
    --> True:
      Predict {1: 2, 0: 8}
    --> False:
      Predict {1: 1, 0: 9}
  --> False:
    Is Humidity == high?
    --> True:
      Predict {1: 5, 0: 5}
    --> False:
      Predict {1: 3, 0: 7}

Predicting the Class

In [0]:
def classify(row, node):
    """See the 'rules of recursion' above."""

    # Base case: we've reached a leaf
    if isinstance(node, Leaf):
        return node.predictions

    # Decide whether to follow the true-branch or the false-branch.
    # Compare the feature / value stored in the node,
    # to the example we're considering.
    if node.question.match(row):
        return classify(row, node.true_branch)
    else:
        return classify(row, node.false_branch)

In [23]:
classify(test_set[11], my_tree)

NameError: ignored

In [0]:
def print_leaf(counts):
    """A nicer way to print the predictions at a leaf."""
    total = sum(counts.values()) * 1.0
    probs = {}
    for lbl in counts.keys():
        probs[lbl] = str(int(counts[lbl] / total * 100)) + "%"
    return probs

In [0]:
#print_leaf(classify(train_set[], my_tree))

In [0]:
for row in test_set:
    print ("Actual: %s. Predicted: %s" %
           (row[-1], print_leaf(classify(row, my_tree))))

Actual: 4.0. Predicted: {4.0: '85%', 2.0: '14%'}
Actual: 2.0. Predicted: {2.0: '97%', 4.0: '2%'}
Actual: 4.0. Predicted: {4.0: '85%', 2.0: '14%'}
Actual: 4.0. Predicted: {4.0: '85%', 2.0: '14%'}
Actual: 2.0. Predicted: {2.0: '97%', 4.0: '2%'}
Actual: 2.0. Predicted: {2.0: '97%', 4.0: '2%'}
Actual: 4.0. Predicted: {4.0: '85%', 2.0: '14%'}
Actual: 2.0. Predicted: {2.0: '97%', 4.0: '2%'}
Actual: 2.0. Predicted: {2.0: '97%', 4.0: '2%'}
Actual: 4.0. Predicted: {4.0: '85%', 2.0: '14%'}
Actual: 4.0. Predicted: {4.0: '85%', 2.0: '14%'}
Actual: 2.0. Predicted: {2.0: '97%', 4.0: '2%'}
Actual: 2.0. Predicted: {2.0: '97%', 4.0: '2%'}
Actual: 2.0. Predicted: {2.0: '97%', 4.0: '2%'}
Actual: 2.0. Predicted: {2.0: '97%', 4.0: '2%'}
Actual: 2.0. Predicted: {2.0: '97%', 4.0: '2%'}
Actual: 4.0. Predicted: {4.0: '85%', 2.0: '14%'}
Actual: 4.0. Predicted: {4.0: '85%', 2.0: '14%'}
Actual: 4.0. Predicted: {4.0: '85%', 2.0: '14%'}
Actual: 4.0. Predicted: {4.0: '85%', 2.0: '14%'}
Actual: 2.0. Predicted: {2.0: 